In [1]:
#using Revise 
#includet("../src/thermal_modeling.jl")
using Flux, DataFrames, CSV, ProgressMeter, Statistics
#using thermal_modeling: TNNCell

In [2]:
# Topology Definitions
mutable struct HeatTransferLayer{U,V,T}
    n_temps::Int
    n_targets::Int
    conductance_net::Dense{U,Matrix{V},Vector{T}}
    adj_mat::Matrix{Int8}
end

function HeatTransferLayer(n_input::Integer, n_temps::Integer, n_targets::Integer)
    # populate adjacency matrix
    adj_mat = zeros(Int8, n_temps, n_temps)
    k = 1
    for col_j in 1:n_temps
        for row_i in col_j + 1:n_temps
            adj_mat[row_i, col_j] = k
            k += 1
        end
    end
    adj_mat = adj_mat + adj_mat'
    n_conds = Int(0.5 * n_temps * (n_temps - 1))
    HeatTransferLayer(n_temps, n_targets,
                      Dense(n_input + n_targets, n_conds, σ),
                      adj_mat)
end

# overload struct to make it callable
function (m::HeatTransferLayer)(all_input)
    n_temps = m.n_temps
    prev_out = @view all_input[1:m.n_targets, :]
    temps = @view all_input[1:n_temps, :]
    
    conductances = m.conductance_net(all_input)
    
    # subtract, scale, and sum
    tmp = hcat([sum(temps[j, :] .- prev_out[i, :] .* conductances[m.adj_mat[i, j], :] 
                for j in 1:n_temps if j != i) 
                    for i in 1:m.n_targets]...)'
    # mutating arrays not allowed in zygote
    """tmp = zeros(eltype(prev_out), size(prev_out))
    for i in 1:m.n_targets
        for j in 1:n_temps
            if j != i
                @. tmp[i, :] += (temps[j, :] - prev_out[i, :]) * conductances[m.adj_mat[i, j], :]
            end
        end
    end"""


    return tmp
end

# specify what is trainable 
Flux.@functor HeatTransferLayer (conductance_net,)

mutable struct TNNCell{U <: Chain,V <: Real,S}
    sample_time::V
    ploss_net::U
    heat_net::HeatTransferLayer
    caps::Vector{V}
    prll::Parallel  # will be defined in inner constructor (no outer definition)
    state0::S
    function TNNCell(sample_time::V, ploss_net::U, heat_net::HeatTransferLayer, caps::Vector{V}, init_hidden::S) where {U <: Chain,V <: Real,S}
        new{U,V,S}(sample_time, ploss_net, heat_net, caps, Parallel(+, ploss_net, heat_net), init_hidden)
    end
end


function TNNCell(n_input::U, n_temps::U, n_targets::U, init_hidden::S) where {U <: Integer,S}
    ploss_net = Chain(Dense(n_input + n_targets, 8, σ),
                      Dense(8, n_targets, σ))
    heat_transfer = HeatTransferLayer(n_input, n_temps, n_targets)
    caps = 0.5f0 .* randn(Float32, n_targets) .- 3f0  # Gaussian mean=-3 std=0.5
    TNNCell(Float32(0.5), ploss_net, heat_transfer, caps, init_hidden)
end

function (m::TNNCell)(prev_̂y, x)
    x_non_temps, x_temps = x
    xx = vcat(prev_̂y, x_temps, x_non_temps)
    rh_ode = m.prll(xx)
    y = prev_̂y .+ m.sample_time .* 10f0.^m.caps .* rh_ode
    return y, prev_̂y
end

# specify what is trainable 
Flux.@functor TNNCell (ploss_net, heat_net, caps)

In [ ]:
const n_input_temps = 2
const n_input_non_temps = 3
const n_total_inputs = n_input_temps + n_input_non_temps
const n_targets = 3
const n_temps = n_targets + n_input_temps
const n_profiles = 49

# smoke-test the topology
xs = [(rand(Float32, n_input_non_temps, n_profiles), 
        rand(Float32, n_input_temps, n_profiles)) for i in 1:10]
h = rand(Float32, n_targets, n_profiles)  # initial hidden state

m = Flux.Recur(TNNCell(n_input_non_temps+n_input_temps, n_temps, n_targets, h), h)

# predict
ys = [m(x) for x in xs]
ys[1]

In [ ]:

ys = [rand(Float32, n_targets, n_profiles) for i in 1:10]
loss(x_l, y_l) = Statistics::mean(Flux.Losses.mse(m(x), y) for (x, y) in zip(x_l, y_l))


In [11]:
function load_dataset(path::String)::DataFrame
    data = CSV.File(path) |> DataFrame
    # FE
    @. data[!, :i_norm] = sqrt(data.i_d^2 + data.i_q^2)
    data[!, :fe1] =
        data.i_norm / maximum(data.i_norm) .* data.motor_speed / maximum(data.motor_speed)
    data
end;

mutable struct TemperatureDataSet
    train_tnsr::Tuple{Any, Vararg{Any}}
    val_tnsr::Tuple{Any, Vararg{Any}}
    test_tnsr::Tuple{Any, Vararg{Any}}
    non_temp_cols::Vector{String}
    temp_cols::Vector{String}
    inp_temp_cols::Vector{String}
    target_cols::Vector{String}

    function TemperatureDataSet(
        data_df::DataFrame,
        p_id::String,
        test_ids::Vector{Int},
        val_ids::Vector{Int},
        target_cols::Vector{String},
        tbptt_len::Int,
    )
        gdf = groupby(data_df, p_id)
        p_sizes = combine(gdf, nrow)
        max_len_test = maximum(filter(:profile_id => n -> n in test_set_pids, p_sizes).nrow)
        max_len_train = maximum(filter(:profile_id => n -> n ∉ test_set_pids, p_sizes).nrow)

        n_test_profiles = length(test_set_pids)
        n_train_profiles = length(keys(gdf)) - n_test_profiles

        c_input_temps = ["ambient", "coolant"]
        c_non_temps = [
            c for
            c in names(data_df) if c ∉ [target_cols..., c_input_temps..., "profile_id"]
        ]

        # create placeholders
        train_tensor_non_temp_x =
            zeros(Float32, (max_len_train, length(c_non_temps), n_train_profiles))
        train_tensor_temps_x =
            zeros(Float32, (max_len_train, length(c_input_temps), n_train_profiles))
        train_tensor_y =
            zeros(Float32, (max_len_train, length(target_cols), n_train_profiles))
        train_sample_weights = zeros(Float32, (max_len_train, n_train_profiles))

        test_tensor_non_temp_x =
            zeros(Float32, (max_len_test, length(c_non_temps), n_test_profiles))
        test_tensor_temps_x =
            zeros(Float32, (max_len_test, length(c_input_temps), n_test_profiles))
        test_tensor_y = zeros(Float32, (max_len_test, length(target_cols), n_test_profiles))
        test_sample_weights = zeros(Float32, (max_len_test, n_test_profiles))

        # fill placeholders
        test_p_idx = 0
        train_p_idx = 0
        @showprogress 0.5 "Computing " for (pid, df) in pairs(gdf)
            if pid.profile_id ∈ test_set_pids
                test_p_idx += 1
                test_tensor_non_temp_x[1:nrow(df), :, test_p_idx] .= df[:, c_non_temps]
                test_tensor_temps_x[1:nrow(df), :, test_p_idx] .= df[:, c_input_temps]
                test_tensor_y[1:nrow(df), :, test_p_idx] .= df[:, target_cols]
                test_sample_weights[1:nrow(df), test_p_idx] .= 1
            else
                train_p_idx += 1
                train_tensor_non_temp_x[1:nrow(df), :, train_p_idx] .= df[:, c_non_temps]
                train_tensor_temps_x[1:nrow(df), :, train_p_idx] .= df[:, c_input_temps]
                train_tensor_y[1:nrow(df), :, train_p_idx] .= df[:, target_cols]
                train_sample_weights[1:nrow(df), train_p_idx] .= 1
            end
        end

        tbptt_len = 128

        train_vec_temps_x =
            [train_tensor_temps_x[i, :, :] for i = 1:size(train_tensor_temps_x, 1)]
        train_vec_non_temp_x =
            [train_tensor_non_temp_x[i, :, :] for i = 1:size(train_tensor_non_temp_x, 1)]
        train_vec_x = collect(zip(train_vec_non_temp_x, train_vec_temps_x))
        train_vec_y = [train_tensor_y[i, :, :] for i = 1:size(train_tensor_y, 1)]
        train_vec_sample_weights =
            [train_sample_weights[i, :] for i = 1:size(train_sample_weights, 1)]

        train_vec_chunked_x = []
        train_vec_chunked_y = []
        train_vec_chunked_w = []

        i = 0
        while i * tbptt_len <= max_len_train
            push!(
                train_vec_chunked_x,
                train_vec_x[i*tbptt_len+1:minimum((
                    (i + 1) * tbptt_len + 1,
                    max_len_train,
                ))],
            )
            push!(
                train_vec_chunked_y,
                train_vec_y[i*tbptt_len+1:minimum((
                    (i + 1) * tbptt_len + 1,
                    max_len_train,
                ))],
            )
            push!(
                train_vec_chunked_w,
                train_vec_sample_weights[i*tbptt_len+1:minimum((
                    (i + 1) * tbptt_len + 1,
                    max_len_train,
                ))],
            )
            i += 1
        end

        test_vec_temps_x =
            [test_tensor_temps_x[i, :, :] for i = 1:size(test_tensor_temps_x, 1)]
        test_vec_non_temp_x =
            [test_tensor_non_temp_x[i, :, :] for i = 1:size(test_tensor_non_temp_x, 1)]
        test_vec_x = collect(zip(train_vec_non_temp_x, test_vec_temps_x))
        test_vec_y = [test_tensor_y[i, :, :] for i = 1:size(test_tensor_y, 1)]
        test_vec_sample_weights =
            [test_sample_weights[i, :] for i = 1:size(test_sample_weights, 1)]

        # TODO
        # val_vec creation and chunking missing
        # test_vec chunking missing
        new(
            (train_vec_chunked_x, train_vec_chunked_y, train_vec_chunked_w),
            (zeros(1), zeros(1), zeros(1)), # not implemented yet
            (zeros(1), zeros(1), zeros(1)), # not implemented yet
            c_non_temps,
            [target_cols; c_input_temps],
            [c_non_temps; c_input_temps],
            target_cols,
        )
    end
end;

function create_TNNCell_from_data(tnsrs::TemperatureDataSet)
    init_hidden = tnsrs.train_tnsr[2][1]
    m = Flux.Recur(
        TNNCell(
            length(tnsrs.non_temp_cols) + length(tnsrs.inp_temp_cols),
            length(tnsrs.temp_cols),
            length(tnsrs.target_cols),
            init_hidden,
        ),
        init_hidden,
    )
end;

function get_data_tup(tnsrs::TemperatureDataSet)
    return tnsrs.train_tnsr
end;


In [ ]:
# fill in DataFrame information
test_p_idx = 0
train_p_idx = 0
@showprogress 0.5 "Computing " for (pid, df) in pairs(gdf)
    if pid.profile_id ∈ test_set_pids
        test_p_idx += 1
        test_tensor_non_temp_x[1:nrow(df), :, test_p_idx] .= df[:, c_non_temps]
        test_tensor_temps_x[1:nrow(df), :, test_p_idx] .= df[:, c_input_temps]
        test_tensor_y[1:nrow(df), :, test_p_idx] .= df[:, target_cols]
        test_sample_weights[1:nrow(df), test_p_idx] .= 1
    else
        train_p_idx += 1
        train_tensor_non_temp_x[1:nrow(df), :, train_p_idx] .= df[:, c_non_temps]
        train_tensor_temps_x[1:nrow(df), :, train_p_idx] .= df[:, c_input_temps]
        train_tensor_y[1:nrow(df), :, train_p_idx] .= df[:, target_cols]
        train_sample_weights[1:nrow(df), train_p_idx] .= 1
    end
end

In [ ]:
tbptt_len = 128

train_vec_temps_x = [train_tensor_temps_x[i, :, :] for i in 1:size(train_tensor_temps_x, 1)]
train_vec_non_temp_x = [train_tensor_non_temp_x[i, :, :] for i in 1:size(train_tensor_non_temp_x, 1)]
train_vec_x = collect(zip(train_vec_non_temp_x, train_vec_temps_x))
train_vec_y = [train_tensor_y[i, :, :] for i in 1:size(train_tensor_y, 1)]
train_vec_sample_weights = [train_sample_weights[i, :] for i in 1:size(train_sample_weights, 1)]

train_vec_chunked_x = []
train_vec_chunked_y = []
train_vec_chunked_w = []

i = 0;
while i*tbptt_len <= max_len_train
    push!(train_vec_chunked_x, train_vec_x[i*tbptt_len+1:minimum(((i+1)*tbptt_len+1, max_len_train))])
    push!(train_vec_chunked_y, train_vec_y[i*tbptt_len+1:minimum(((i+1)*tbptt_len+1, max_len_train))])
    push!(train_vec_chunked_w, train_vec_sample_weights[i*tbptt_len+1:minimum(((i+1)*tbptt_len+1, max_len_train))])
    global i+= 1
end

test_vec_temps_x = [test_tensor_temps_x[i, :, :] for i in 1:size(test_tensor_temps_x, 1)]
test_vec_non_temp_x = [test_tensor_non_temp_x[i, :, :] for i in 1:size(test_tensor_non_temp_x, 1)]
test_vec_x = collect(zip(train_vec_non_temp_x, test_vec_temps_x))
test_vec_y = [test_tensor_y[i, :, :] for i in 1:size(test_tensor_y, 1)]
test_vec_sample_weights = [test_sample_weights[i, :] for i in 1:size(test_sample_weights, 1)];


In [7]:
using Statistics: mean
n_epochs = 100
pbar = Progress(n_epochs, desc="Training Epochs", start=1, showspeed=true)
init_hidden = train_vec_y[1]
m = Flux.Recur(TNNCell(length(c_non_temps)+length(c_input_temps),
                       length(c_temps),
                       length(target_cols),
                       init_hidden),
               init_hidden)
ps = params(m)
opt = ADAM(1e-3)

function sample_weighted_loss(x::Vector{Tuple{Matrix{T}, Matrix{T}}}, y::Vector{U}, w::Vector{V}) where {T, U, V}
   mean(Flux.Losses.mse(m(xi), yi, agg=z->mean(wi' .* z)) for (xi, yi, wi) in zip(x, y, w)) 
end

UndefVarError: UndefVarError: train_vec_y not defined

In [ ]:
# training
data_tup = zip(train_vec_chunked_x, train_vec_chunked_y, train_vec_chunked_w);
for epoch in 1:n_epochs
    Flux.reset!(m)
    Flux.train!(sample_weighted_loss, ps, data_tup, opt)
    next!(pbar, showvalues = [(:epoch, epoch)])
end



In [15]:
using BenchmarkTools


In [ ]:
@benchmark Flux.train!($sample_weighted_loss, $ps, $data_tup, $opt)

In [19]:
test_set_pids = [60, 62, 74]
target_cols = ["pm", "stator_tooth", "stator_winding", "stator_yoke"]
ds_df = load_dataset("/home/wilhelmk/dev/projects/datasets/kaggle_emotor_temps.csv")
ds = TemperatureDataSet(ds_df, "profile_id", test_set_pids, [0, 0], target_cols, 128)
m = create_TNNCell_from_data(ds)
ps = params(m);
opt = ADAM(1e-3)

function sample_weighted_loss(
    x::Vector{Tuple{Matrix{T},Matrix{T}}},
    y::Vector{U},
    w::Vector{V},
) where {T,U,V}
    mean(
        Flux.Losses.mse(m(xi), yi, agg = z -> mean(wi' .* z)) for
        (xi, yi, wi) in zip(x, y, w)
    )
end;


Computing   9%|███▋                                     |  ETA: 0:00:06

Computing  19%|███████▊                                 |  ETA: 0:00:05

Computing  28%|███████████▎                             |  ETA: 0:00:04

Computing  38%|███████████████▌                         |  ETA: 0:00:04

Computing  48%|███████████████████▋                     |  ETA: 0:00:03

Computing  65%|██████████████████████████▊              |  ETA: 0:00:02

Computing  78%|████████████████████████████████▏        |  ETA: 0:00:01

Computing  88%|████████████████████████████████████▎    |  ETA: 0:00:01

Computing 100%|█████████████████████████████████████████| Time: 0:00:05


In [21]:
@benchmark Flux.train!($sample_weighted_loss, $ps, $zip(ds.train_tnsr...), $opt)

ArgumentError: ArgumentError: number of columns of each array must match (got (1, 66, 66))

In [ ]:
function train_one_epoch()
    Flux.train!($sample_weighted_loss, $ps, $data_tup, $opt)
end

In [ ]:
@benchmark train_one_epoch()